In [16]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession, DataFrame

In [17]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/17 10:31:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
time_offset = +1

if time_offset >= 0:
    time_offset_str = f"+{abs(time_offset):02d}:00"
else:
    time_offset_str = f"-{abs(time_offset):02d}:00"

datetime_format = "yyyy-MM-dd HH:mm"
windows_size = "1 hour"

spark.conf.set('spark.sql.session.timeZone', time_offset_str)

In [19]:
data_df = spark.createDataFrame([
    (1, "2023-03-25 23:00"),
    (1, "2023-03-26 00:00"),
    (1, "2023-03-26 01:00"),
    (1, "2023-03-26 01:59"),
    # (1, "2023-03-26 02:00"),
    (1, "2023-03-26 03:00"),
    (1, "2023-03-26 04:00"),
    (1, "2023-03-26 10:00"),
], schema=["id", "datetime"])

data_df.show(truncate=False)

+---+----------------+
|id |datetime        |
+---+----------------+
|1  |2023-03-25 23:00|
|1  |2023-03-26 00:00|
|1  |2023-03-26 01:00|
|1  |2023-03-26 01:59|
|1  |2023-03-26 03:00|
|1  |2023-03-26 04:00|
|1  |2023-03-26 10:00|
+---+----------------+



In [21]:
f"{-time_offset} hours"

'-1 hours'

In [30]:
windows_size = "2 hour"

In [33]:
data1_df = data_df.withColumn("window", F.window(timeColumn=F.col("datetime"), windowDuration=windows_size))
data1_df = data1_df.withColumn("shifted_window", F.window(timeColumn=F.col("datetime"), windowDuration=windows_size, startTime=f"-120 minute"))
data1_df.show(truncate=False)

AnalysisException: [DATATYPE_MISMATCH.PARAMETER_CONSTRAINT_VIOLATION] Cannot resolve "window(datetime, 7200000000, 7200000000, -7200000000)" due to data type mismatch: The `abs(start_time)`(7200000000L) must be < the `slide_duration`(7200000000L).;
'Project [id#0L, datetime#1, window#200, window(cast(datetime#1 as timestamp), 7200000000, 7200000000, -7200000000) AS shifted_window#206]
+- Project [id#0L, datetime#1, window#201 AS window#200]
   +- Project [named_struct(start, knownnullable(precisetimestampconversion(((precisetimestampconversion(cast(datetime#1 as timestamp), TimestampType, LongType) - CASE WHEN (((precisetimestampconversion(cast(datetime#1 as timestamp), TimestampType, LongType) - 0) % 7200000000) < cast(0 as bigint)) THEN (((precisetimestampconversion(cast(datetime#1 as timestamp), TimestampType, LongType) - 0) % 7200000000) + 7200000000) ELSE ((precisetimestampconversion(cast(datetime#1 as timestamp), TimestampType, LongType) - 0) % 7200000000) END) - 0), LongType, Ti

In [9]:
data1_df = data1_df.withColumn("window_start_utc", F.col("window_utc").start)
data1_df = data1_df.withColumn("window_end_utc", F.col("window_utc").end)
data1_df.show(truncate=False)

+---+----------+----------------+------------------------------------------+-------------------+-------------------+
|id |unix_time |datetime_utc    |window_utc                                |window_start_utc   |window_end_utc     |
+---+----------+----------------+------------------------------------------+-------------------+-------------------+
|1  |1679781600|2023-03-25 22:00|{2023-03-25 22:00:00, 2023-03-25 23:00:00}|2023-03-25 22:00:00|2023-03-25 23:00:00|
|1  |1679785200|2023-03-25 23:00|{2023-03-25 23:00:00, 2023-03-26 00:00:00}|2023-03-25 23:00:00|2023-03-26 00:00:00|
|1  |1679788800|2023-03-26 00:00|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|2023-03-26 00:00:00|2023-03-26 01:00:00|
|1  |1679792340|2023-03-26 00:59|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|2023-03-26 00:00:00|2023-03-26 01:00:00|
|1  |1679792400|2023-03-26 01:00|{2023-03-26 01:00:00, 2023-03-26 02:00:00}|2023-03-26 01:00:00|2023-03-26 02:00:00|
|1  |1679796000|2023-03-26 02:00|{2023-03-26 02:00:00, 2023-03-2

In [12]:
data1_df = data1_df.withColumn("window_start", F.from_utc_timestamp(F.col("window_start_utc"), time_zone))
data1_df = data1_df.withColumn("window_end", F.from_utc_timestamp(F.col("window_end_utc"), time_zone))
data1_df.show(truncate=False)

+---+----------+----------------+------------------------------------------+-------------------+-------------------+-------------------+-------------------+
|id |unix_time |datetime_utc    |window_utc                                |window_start_utc   |window_end_utc     |window_start       |window_end         |
+---+----------+----------------+------------------------------------------+-------------------+-------------------+-------------------+-------------------+
|1  |1679781600|2023-03-25 22:00|{2023-03-25 22:00:00, 2023-03-25 23:00:00}|2023-03-25 22:00:00|2023-03-25 23:00:00|2023-03-25 23:00:00|2023-03-26 00:00:00|
|1  |1679785200|2023-03-25 23:00|{2023-03-25 23:00:00, 2023-03-26 00:00:00}|2023-03-25 23:00:00|2023-03-26 00:00:00|2023-03-26 00:00:00|2023-03-26 01:00:00|
|1  |1679788800|2023-03-26 00:00|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|2023-03-26 00:00:00|2023-03-26 01:00:00|2023-03-26 01:00:00|2023-03-26 03:00:00|
|1  |1679792340|2023-03-26 00:59|{2023-03-26 00:00:00, 202

23/07/14 20:16:00 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 961534 ms exceeds timeout 120000 ms
23/07/14 20:16:00 WARN SparkContext: Killing executors is not supported by current scheduler.
23/07/14 20:31:18 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B

## Appunti

- Ho capito che bisogna fare il bucketing impostando la time zone a UTC altrimenti i bucket vengono fuori non uniformi
- PySpark non ha DateTimeType che includano le TimeZone
- PySpark ha una TimeZone impostata di default e per ogni DateTimeType che ha utilizza implicitamente quella